In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Convert COCO dataset to SAM format

In [ ]:
# Asegúrate de instalar las bibliotecas necesarias
!pip install opencv-python-headless pillow pycocotools

import json
import os
import shutil
import numpy as np
import pandas as pd
from PIL import Image
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
from google.colab import files

# Crear carpetas para guardar las imágenes y las máscaras
os.makedirs('Breast/images', exist_ok=True)
os.makedirs('Breast/masks', exist_ok=True)

def process_annotations(json_file, img_dir, csv_file):
    # Cargar el archivo JSON de COCO
    coco = COCO(json_file)

    data = []

    # Procesar las anotaciones y generar las máscaras
    for img_id in coco.getImgIds():
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(img_dir, img_info['file_name'])
        image = Image.open(img_path).convert("RGB")

        # Copiar la imagen a la carpeta 'Breast/images'
        new_img_path = f'Breast/images/{img_info["file_name"]}'
        shutil.copy(img_path, new_img_path)

        # Crear una máscara vacía con el mismo tamaño que la imagen
        mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

        # Obtener las anotaciones para la imagen
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        for ann in anns:
            if 'segmentation' in ann:
                rle = coco.annToRLE(ann)
                decoded_mask = maskUtils.decode(rle)
                mask = np.maximum(mask, decoded_mask)

        # Convertir la máscara a una imagen binaria (blanco y negro)
        mask = (mask > 0).astype(np.uint8) * 255

        # Guardar la máscara en la carpeta 'Breast/masks'
        mask_img = Image.fromarray(mask)
        mask_filename = os.path.splitext(img_info['file_name'])[0] + '.png'
        mask_path = f'Breast/masks/{mask_filename}'
        mask_img.save(mask_path)

        # Agregar las rutas al DataFrame
        data.append([new_img_path, f'/{mask_path}'])

    # Guardar las rutas en un archivo CSV sin cabeceras
    df = pd.DataFrame(data)
    df.to_csv(csv_file, index=False, header=False)

# Procesar las anotaciones de train y val y generar los CSV
process_annotations('/content/drive/MyDrive/1data/data/train.json', '/content/drive/MyDrive/1data/data/train', 'train.csv')
process_annotations('/content/drive/MyDrive/1data/data/valid.json', '/content/drive/MyDrive/1data/data/valid', 'valid.csv')

print("Conversión completa y archivos CSV generados.")


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Conversión completa y archivos CSV generados.


# If you are in collab

In [ ]:
import shutil
import os

# Ruta de la carpeta que deseas comprimir
folder_to_compress = '/content/Breast'
# Nombre del archivo ZIP de salida
output_zip_file = '/content/Breast.zip'

# Comprimir la carpeta
shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_compress)

print(f'Carpeta comprimida en: {output_zip_file}')


Carpeta comprimida en: /content/Breast.zip


In [ ]:
from google.colab import files

# Descargar el archivo ZIP
files.download(output_zip_file)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>